# 데이터 재구조화

과연 우리가 탐색하려는 데이터가 제대로 된 것인가? 데이터가 분석이나 시각화를 위해 필요한 데이터 구조로 딱 맞아떨어지지 않는 경우가 굉장히 많음. 이때 필요한 것이 데이터를 분석 목적, 기법에 맞게 자유자재로 변형하여 재구조화하는 일! `melt()`, `pivot()`, `merge()`를 사용한다. `pandas` 라이브러리가 필요하다.

In [1]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 생성
np.random.seed(101)
a = 101
b = 2
c = 50

ID = np.repeat(np.arange(1, a + 1), repeats=b)
time = np.tile(np.arange(1, b + 1), a)
values = np.random.normal(size=a * b * c).reshape(a * b, c)

# 데이터 프레임 생성
data_matrix = np.column_stack([ID, time, values])
column_names = ['ID', 'time'] + [f'{letter}.{num}' for letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'[:25] for num in range(1, 3)]
datas1 = pd.DataFrame(data_matrix, columns=column_names)

# 데이터 구조 확인
datas1.info()
datas1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 52 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      202 non-null    float64
 1   time    202 non-null    float64
 2   A.1     202 non-null    float64
 3   A.2     202 non-null    float64
 4   B.1     202 non-null    float64
 5   B.2     202 non-null    float64
 6   C.1     202 non-null    float64
 7   C.2     202 non-null    float64
 8   D.1     202 non-null    float64
 9   D.2     202 non-null    float64
 10  E.1     202 non-null    float64
 11  E.2     202 non-null    float64
 12  F.1     202 non-null    float64
 13  F.2     202 non-null    float64
 14  G.1     202 non-null    float64
 15  G.2     202 non-null    float64
 16  H.1     202 non-null    float64
 17  H.2     202 non-null    float64
 18  I.1     202 non-null    float64
 19  I.2     202 non-null    float64
 20  J.1     202 non-null    float64
 21  J.2     202 non-null    float64
 22  K.

,ID,time,A.1,A.2,B.1,B.2,C.1,C.2,D.1,D.2,...,U.1,U.2,V.1,V.2,W.1,W.2,X.1,X.2,Y.1,Y.2
0,1.0,1.0,2.706850,0.628133,0.907969,0.503826,0.651118,-0.319318,-0.848077,0.605965,...,-0.993263,0.196800,-1.136645,0.000366,1.025984,-0.156598,-0.031579,0.649826,2.154846,-0.610259
1,1.0,2.0,-0.755325,-0.346419,0.147027,-0.479448,0.558769,1.024810,-0.925874,1.862864,...,-0.568581,1.407338,0.641806,-0.905100,-0.391157,1.028293,-1.972605,-0.866885,0.720788,-1.223082
2,2.0,1.0,1.606780,-1.115710,-1.385379,-1.329660,0.041460,-0.411055,-0.771329,0.110477,...,0.327845,0.674485,-0.174057,0.780140,-0.383258,-0.409318,0.343539,0.196275,-0.982776,2.231555
3,2.0,2.0,-0.971393,-1.522333,1.133703,0.528187,0.393461,-0.630507,-1.398290,-0.219311,...,-1.235614,-0.798428,0.754121,-0.772041,-0.034432,0.688408,0.081523,-0.415191,0.053661,-2.115113
4,3.0,1.0,-0.279510,1.062710,1.752014,0.695547,0.153661,0.167638,-0.765930,0.962299,...,-1.409126,0.870907,1.585812,0.929658,-0.550876,0.942045,-0.975349,-1.054851,2.165421,-1.267240


이 데이터의 구조를 보자.

-   ID : 1,2,3으로 구성
-   time : 1교시, 2교시
-   A.1 = A반 1번 문항 점수

데이터들의 변수가 너무 많다. (알파벳별, 숫자별로 쪼개짐.)

따라서 이 데이터는 클러스터링하기 적절하지 않으며, 재구조화가 필요하다.

## melt & pivot 이용하기

-   `melt()`를 이용해 ID와 time 외 모든 변수들을 합쳐보자.
-   남겨놓을 변수는 `id_vars`에 저장한다.

In [2]:
# melt() 사용하기
datas2 = pd.melt(datas1, id_vars=['ID', 'time'])
print(datas2.head())

    ID  time variable     value
0  1.0   1.0      A.1  2.706850
1  1.0   2.0      A.1 -0.755325
2  2.0   1.0      A.1  1.606780
3  2.0   2.0      A.1 -0.971393
4  3.0   1.0      A.1 -0.279510


이제, `pivot()`을 이용해 다시 데이터를 원래대로 돌려보자.

-   `pivot_table()`을 사용하여 `ID`와 `time`을 기준으로 변환합니다.
-   `reset_index()`로 인덱스를 재설정합니다.

In [3]:
# pivot()을 이용해 원래 데이터로 변환
datas3 = datas2.pivot_table(index=['ID', 'time'], columns='variable', values='value').reset_index()
datas3.columns.name = None  # Remove the name of the columns
print(datas3.head())

# 데이터 일치 확인
print(pd.testing.assert_frame_equal(datas1, datas3))
print(datas1.describe().subtract(datas3.describe()).abs().sum())

    ID  time       A.1       A.2       B.1       B.2       C.1       C.2  \
0  1.0   1.0  2.706850  0.628133  0.907969  0.503826  0.651118 -0.319318   
1  1.0   2.0 -0.755325 -0.346419  0.147027 -0.479448  0.558769  1.024810   
2  2.0   1.0  1.606780 -1.115710 -1.385379 -1.329660  0.041460 -0.411055   
3  2.0   2.0 -0.971393 -1.522333  1.133703  0.528187  0.393461 -0.630507   
4  3.0   1.0 -0.279510  1.062710  1.752014  0.695547  0.153661  0.167638   

        D.1       D.2  ...       U.1       U.2       V.1       V.2       W.1  \
0 -0.848077  0.605965  ... -0.993263  0.196800 -1.136645  0.000366  1.025984   
1 -0.925874  1.862864  ... -0.568581  1.407338  0.641806 -0.905100 -0.391157   
2 -0.771329  0.110477  ...  0.327845  0.674485 -0.174057  0.780140 -0.383258   
3 -1.398290 -0.219311  ... -1.235614 -0.798428  0.754121 -0.772041 -0.034432   
4 -0.765930  0.962299  ... -1.409126  0.870907  1.585812  0.929658 -0.550876   

        W.2       X.1       X.2       Y.1       Y.2  
0 -0.156

## pivot_table의 응용

문항별 평균 점수를 만들어보자.

In [4]:
# pivot_table()을 이용한 문항별 평균 점수
mean_scores = datas2.pivot_table(index='time', columns='variable', values='value', aggfunc='mean')
print(mean_scores)

# groupby()를 이용한 A.1의 문항별 평균 점수
mean_A1 = datas1.groupby('time')['A.1'].mean()
print(mean_A1)

variable       A.1       A.2       B.1       B.2       C.1       C.2  \
time                                                                   
1.0       0.072739  0.005211 -0.053658 -0.036618 -0.143627 -0.149684   
2.0      -0.054403 -0.122057 -0.104332  0.032131  0.077476 -0.099547   

variable       D.1       D.2       E.1       E.2  ...       U.1       U.2  \
time                                              ...                       
1.0      -0.069464  0.201605 -0.144862  0.099348  ... -0.122329  0.120573   
2.0       0.167639 -0.009080  0.159003 -0.053176  ...  0.071788 -0.091255   

variable       V.1       V.2       W.1       W.2       X.1       X.2  \
time                                                                   
1.0       0.077454 -0.075341 -0.124916  0.014511 -0.137549  0.076429   
2.0      -0.054019 -0.062990  0.063584 -0.052611  0.142775  0.062823   

variable       Y.1       Y.2  
time                          
1.0       0.109873  0.153845  
2.0       0.125529  

## merge() : 두 데이터프레임 합치기

이번에도 새 데이터프레임을 만들어보자.

In [5]:
# 새 데이터프레임 생성
v = 5
x2 = np.column_stack([np.repeat(np.arange(1, a - v + 1), repeats=b), 
                      np.tile(np.arange(1, b + 1), a - v),
                      np.random.normal(size=(a - v) * b * 2).reshape((a - v) * b, 2)])
datas_svy = pd.DataFrame(x2, columns=['ID', 'time', 'HT', 'WT'])
datas_svy.head()

,ID,time,HT,WT
0,1.0,1.0,0.420891,1.463235
1,1.0,2.0,0.960482,-0.521592
2,2.0,1.0,1.383678,2.259198
3,2.0,2.0,-1.183410,0.789278
4,3.0,1.0,-1.399660,-0.357220


만든 새 데이터프레임 `datas_svy`를 `datas1`과 비교해보자.

확연히 다른 구조를 가지고 있을 것이다.

In [6]:
# 두 데이터프레임 구조 비교
print(f'datas1 shape: {datas1.shape}')
print(f'datas_svy shape: {datas_svy.shape}')

datas1 shape: (202, 52)
datas_svy shape: (192, 4)


merge()를 이용해 두 데이터프레임을 합쳐보자.

-   datas1은 1~4번째 변수. datas_svy는 그대로.
-   기준 : ID

In [7]:
# 데이터프레임 병합
merged_1 = pd.merge(datas1[['ID', 'time', 'A.1', 'A.2']], datas_svy, on='ID')
print(merged_1.info())
print(merged_1.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      384 non-null    float64
 1   time_x  384 non-null    float64
 2   A.1     384 non-null    float64
 3   A.2     384 non-null    float64
 4   time_y  384 non-null    float64
 5   HT      384 non-null    float64
 6   WT      384 non-null    float64
dtypes: float64(7)
memory usage: 21.1 KB
None
    ID  time_x       A.1       A.2  time_y        HT        WT
0  1.0     1.0  2.706850  0.628133     1.0  0.420891  1.463235
1  1.0     1.0  2.706850  0.628133     2.0  0.960482 -0.521592
2  1.0     2.0 -0.755325 -0.346419     1.0  0.420891  1.463235
3  1.0     2.0 -0.755325 -0.346419     2.0  0.960482 -0.521592
4  2.0     1.0  1.606780 -1.115710     1.0  1.383678  2.259198


-   datas1은 1~4번째 변수. datas_svy는 그대로.
-   기준 : ID, time, x의 모든 파트 살리기

In [8]:
# 모든 변수 기준으로 병합
merged_2 = pd.merge(datas1[['ID', 'time', 'A.1', 'A.2']], datas_svy, on=['ID', 'time'], how='left')
print(merged_2.info())
print(merged_2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      202 non-null    float64
 1   time    202 non-null    float64
 2   A.1     202 non-null    float64
 3   A.2     202 non-null    float64
 4   HT      192 non-null    float64
 5   WT      192 non-null    float64
dtypes: float64(6)
memory usage: 9.6 KB
None
    ID  time       A.1       A.2        HT        WT
0  1.0   1.0  2.706850  0.628133  0.420891  1.463235
1  1.0   2.0 -0.755325 -0.346419  0.960482 -0.521592
2  2.0   1.0  1.606780 -1.115710  1.383678  2.259198
3  2.0   2.0 -0.971393 -1.522333 -1.183410  0.789278
4  3.0   1.0 -0.279510  1.062710 -1.399660 -0.357220


NA가 있는 열 찾기

NA 값을 포함하는 열의 이름을 찾자.

In [9]:
# NA가 있는 열 찾기
na_columns = merged_2.columns[merged_2.isna().any()].tolist()
print(na_columns)

['HT', 'WT']
